This is a web scraper built to scrape job data from indeed.com.

The scraper uses selenium webdriver to:
    1. Open browser
    2. Search for remote positions in data science
    3. Pull data fields: job title, company, salary, location, date, job_desc
    4. Write data for each role to a jobs list
    5. Paginate to next page and repeat process until no more pages left.
    6. Write jobs data to a csv (jobs.csv)

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import random
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import re
import numpy as np
from datetime import datetime, timedelta
import os
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize, MWETokenizer

options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),options=options) #WebDriver is used to drive a browser (Opens Browser)
driver.get("https://www.indeed.com") #this will navigate to the page
driver.find_element("xpath", '//*[@id="CookiePrivacyNotice"]/div/button').click()
time.sleep(3) #this will suspend execution for 3 seconds
driver.find_element("xpath",'//*[@id="text-input-what"]').send_keys("Data Science") #Driver finds the WHAT entry box and inputs the user input variable job_title
time.sleep(3)
driver.find_element("xpath", '//*[@id="text-input-where"]').send_keys(Keys.CONTROL,"a") #Driver finds the where entry box and deletes everything then inputs user input variable location
driver.find_element("xpath", '//*[@id="text-input-where"]').send_keys(Keys.DELETE)
driver.find_element("xpath",'//*[@id="text-input-where"]').send_keys("Remote")
time.sleep(3)
driver.find_element("xpath",'/html/body/div').click() #driver finds the element # why is this line necessary?
time.sleep(3)
try:
    driver.find_element("xpath", '//*[@id="jobsearch"]/button').click() #driver tries to find the jobsearch submit button
except:
    driver.find_element("xpath", '//*[@id="whatWhereFormId"]/div[3]/button').click()
time.sleep(2)
jobs_list = []
links = []
current_html = driver.page_source
soup = BeautifulSoup(current_html, "html.parser")

is_element_present = True
while is_element_present:
    try:
        driver.find_element("xpath", '//*[@data-testid="pagination-page-next"]')
        for post in soup.select('.job_seen_beacon'):
            try:
                data = {
                    "job_title": post.select('.jobTitle')[0].get_text().strip(),
                    "company": post.select('.companyName')[0].get_text().strip(),
                    "salary": post.select('.attribute_snippet')[0].get_text().strip(),
                    "location": post.select('.companyLocation')[0].get_text().strip(),
                    "date": post.select('.date')[0].get_text().strip(),
                    "links": post.select('a')[0].get('href')
                    }
            except IndexError:
                continue
            jobs_list.append(data)
        df=pd.DataFrame(jobs_list)
        driver.find_element("xpath", '//*[@data-testid="pagination-page-next"]').click()
        time.sleep(random.randint(1, 2))
        soup=BeautifulSoup(driver.page_source, "html.parser")
    except NoSuchElementException:
        for post in soup.select('.job_seen_beacon'):
            try:
                data = {
                    "job_title": post.select('.jobTitle')[0].get_text().strip(),
                    "company": post.select('.companyName')[0].get_text().strip(),
                    "salary": post.select('.attribute_snippet')[0].get_text().strip(),
                    "location": post.select('.companyLocation')[0].get_text().strip(),
                    "date": post.select('.date')[0].get_text().strip(),
                    "links": post.select('a')[0].get('href')
                    }
            except IndexError:
                continue
            jobs_list.append(data)
        df=pd.DataFrame(jobs_list)
        break

descriptions=[]
for link in df.links: 
    linktosearch = "https://www.indeed.com"+link
    driver.get(linktosearch)
    jd = driver.find_element("xpath",'//div[@id="jobDescriptionText"]').text
    descriptions.append(jd)
    time.sleep(random.randint(1, 2))

df['job_desc'] = descriptions

df.to_csv('jobs.csv')

Now that job listings have been scraped into jobs.csv, we should begin cleaning and exploring our dataset. Check job-data-analysis.ipynb